# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# you sit West and hold:
hand = 'KJ7432.9.A.QJT86'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["PAD_START", "PAD_START", "PASS", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1S', 'insta_score': 0.713, 'expected_score': 38, 'adjustment': 36},
 {'call': '2S', 'insta_score': 0.281, 'expected_score': 33, 'adjustment': 14}]

In [4]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS', 'insta_score': 0.942}]

In [5]:
bid.samples

['KJxxxx.9.A.QJT8x Axx.AJ8xxx.KJx.x QT.KTxx.9x.K9xxx 98.Qx.QT8xxxx.Ax 0.21122',
 'KJxxxx.9.A.QJT8x A.AK8xxxx.8x.A9x QT.JTxx.T9xx.xxx 98xx.Q.KQJxxx.Kx 0.15015',
 'KJxxxx.9.A.QJT8x Ax.KQxxxx.Kxx.A9 8x.T8xx.Q9.Kxxxx QT9.AJ.JT8xxxx.x 0.14997',
 'KJxxxx.9.A.QJT8x Axx.AQT8x.KJ.Kxx T9.Jxxxx.xxx.A9x Q8.Kx.QT98xxx.xx 0.14625',
 'KJxxxx.9.A.QJT8x A9.KQT8xx.9x.Axx T8.Axxx.K8x.9xxx Qxx.Jx.QJTxxxx.K 0.14361',
 'KJxxxx.9.A.QJT8x 98.AKJ8xxx.Q8.Ax T.QTx.Txxx.Kxxxx AQxx.xx.KJ9xxx.9 0.14015',
 'KJxxxx.9.A.QJT8x AQ.A8xxxx.Kx.xxx T8x.QJTx.QTx.K9x 9x.Kx.J98xxxx.Ax 0.12611',
 'KJxxxx.9.A.QJT8x A8x.KJTxx.Kx.K9x Tx.AQ8xx.Jxx.xxx Q9.xx.QT98xxx.Ax 0.12165',
 'KJxxxx.9.A.QJT8x ATx.KT8xxx.KQ.Kx 8x.Qxxx.xxx.Axxx Q9.AJ.JT98xxx.9x 0.12052',
 'KJxxxx.9.A.QJT8x Qx.KQJTxx.KJ.K9x A98.A8xx.8xx.xxx Tx.xx.QT9xxxx.Ax 0.11876']

In [6]:
auction = ["PAD_START", "PAD_START", "PASS", "PASS", "1S", "2H", "PASS", "3D", "PASS", "3N" , "PASS", "PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': 'PASS',
  'insta_score': 0.998,
  'expected_score': -91,
  'adjustment': 50},
 {'call': '4S',
  'insta_score': 0.002,
  'expected_score': -478,
  'adjustment': -1433}]

In [7]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

KJxxxx.9.A.QJT8x AQx.JTxxx.xxx.xx x.AQ8x.QT9xx.A9x T98.Kxx.KJ8x.Kxx
KJxxxx.9.A.QJT8x Q8x.JTxxx.Kxx.Kx T.A8xx.QT98x.Axx A9x.KQx.Jxxx.9xx
KJxxxx.9.A.QJT8x QT8.ATxx.Txx.Kxx 9.KQxx.KQ8xx.A9x Axx.J8xx.J9xx.xx
KJxxxx.9.A.QJT8x T9x.AQTx.T9xx.Kx x.Jxxx.KJ8xx.Axx AQ8.K8xx.Qxx.9xx
KJxxxx.9.A.QJT8x A8x.J8xxx.xx.Kxx 9.AKQx.QJT8x.Axx QTx.Txx.K9xxx.9x
KJxxxx.9.A.QJT8x Q8x.JTxxx.9xx.Kx x.AK8x.KQ8xx.A9x AT9.Qxx.JTxx.xxx
KJxxxx.9.A.QJT8x AT9.J8xxx.Kx.xxx Q.ATxx.QJ9xxx.Ax 8xx.KQx.T8xx.K9x
KJxxxx.9.A.QJT8x ATx.KJxxx.Q9x.Ax 9.AQxx.Kxxxx.K9x Q8x.T8x.JT8x.xxx
KJxxxx.9.A.QJT8x QTx.Kxxxx.xx.9xx A.QJ8x.KT9xxxx.K 98x.ATx.QJ8.Axxx
KJxxxx.9.A.QJT8x T8x.AKT8x.xx.A9x Q.xxxx.KJxxx.Kxx A9x.QJx.QT98x.xx
